<a href="https://colab.research.google.com/github/ptmhoang97/process_dataset/blob/main/rename_and_split_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive._mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Choose dataset to unzip
dataset_name = "coco2017_train_truck_VOC"

In [ ]:
dataset_path = "/content/gdrive/MyDrive/_dataset/{0}.zip".format(dataset_name)
dataset_unzip_path = "/content"
!unzip {dataset_path} -d {dataset_unzip_path}

In [4]:
from PIL import Image
import os
import fnmatch

def convert_png_to_jpg(file_location):
    pattern = '*.png'
    image_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            image_path.append(full_path)
    
    if not image_path:
        print("No png images")
    else:
        print("Converting png to jpg ...")
        for path in image_path:
            path_png = '{}'.format(path)
            path_jpg = path_png.replace(".png",".jpg")
            
            # Convert the link with '\' to '\\' so that it can pass to Image.open() 
            path_png = path_png.encode('unicode-escape').decode()
            path_jpg = path_jpg.encode('unicode-escape').decode()

            img_png = Image.open(path_png)
            img_png_rbg = img_png.convert('RGB')
            img_png_rbg.save(path_jpg)
        print("Done converting!")

In [5]:
import os
import fnmatch

def delete_all_png(file_location):
    pattern = '*.png'
    image_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            image_path.append(full_path)

    if not image_path:
        print("No png images")
    else:
        print("Deleting all png ...")
        for images in test:
            if images.endswith(".png"):
                os.remove(os.path.join(file_location, images))
        print("Done deleting!")

In [6]:
import os
import fnmatch
import shutil

def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]]

def filter_files(file_location,type_file):
    pattern = "*.{}".format(type_file)
    paths = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        path = path[:-4]
        paths.append(path)
    return paths

def check_pair_dataset(file_location,type_file_pic,type_file_anno):
    path_jpg = filter_files(file_location,type_file_pic)
    path_txt = filter_files(file_location,type_file_anno)

    for path in path_txt:
        if path == "classes":
            path_txt.remove("classes")
    print("Checking pair dataset ...")
    if returnNotMatches(path_jpg,path_txt)[0]:
        raise Exception("Files " + type_file_pic + ": " + str(returnNotMatches(path_jpg,path_txt)[0]) + " miss " + type_file_anno + " files")
    elif returnNotMatches(path_jpg,path_txt)[1]:
        raise Exception("Files " + type_file_anno + ": " + str(returnNotMatches(path_jpg,path_txt)[1]) + " miss " + type_file_pic + " files")
    else:
        pass
    print("Done checking! No mismtach!")

In [7]:
from PIL import Image
import os
import fnmatch

def rename_file_with_order(file_location,name_file,start_count_number,type_file):
    pattern = "*.{}".format(type_file)
    file_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            file_path.append(full_path)

    temp_count = start_count_number
    
    for path in file_path:
        name = "{0}_{1}".format(name_file,temp_count)
        start = path.rfind("/")
        end = path.rfind(".")

        start_part = path[:start+1]
        end_part = path[end:]
        
        old_name = path
        new_name = start_part + name + end_part
        
        # print("***")
        # print(old_name)
        print(new_name)
        
        os.rename(old_name,new_name)
        temp_count+=1

def rename_picFile_and_annoFile_with_order(file_location,name_file,start_count_number,type_pic_file,type_anno_file):
    print("Renaming files ...")

    # Rename pic file
    rename_file_with_order(file_location,name_file,start_count_number,type_pic_file)  
    # Rename anno file
    rename_file_with_order(file_location,name_file,start_count_number,type_anno_file)

In [35]:
import os, sys
import shutil
import re
import tqdm

import math

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def split_dataset(file_location,scale_train):
    print("Splitting dataset ...")
    paths = {
        'TRAIN_PATH': os.path.join(file_location,"train"),
        'TEST_PATH': os.path.join(file_location,"test"),
    }

    # If using labelimg to create dataset, collect classes from "classes.txt" and remove file.     
    classes = []
    dirs = os.listdir(file_location)
    for file_name in dirs:
        if file_name == "classes.txt":
            full_path = os.path.join(file_location, file_name)
            with open(full_path,"r") as f:
                classes_temp = f.readlines()
                for cls in classes_temp:
                    if cls.endswith("\n"):
                        classes.append(cls[:-1])
                    else:
                        classes.append(cls)
            os.remove(full_path)
    
    dirs = os.listdir(file_location)
    total_dataset = len(dirs)
    #print(total_dataset)
    #print(dirs)
    num_train_dataset = round_up_to_even(total_dataset*scale_train)

    # print("num_train_dataset: " + str(num_train_dataset))
    # print("num_test_dataset: " + str(total_dataset - num_train_dataset))

    for path in paths.values():
        if not os.path.exists(path):
            os.mkdir(path)
            # print("Create: " + path)
    
    try:
        dirs.remove("_darknet.labels")
        dirs.remove("classes.txt")
    except Exception:
        pass
    
    dirs_temp = ['truck_COCO_2121.xml', 'truck_COCO_5771.jpg', 'truck_COCO_2835.jpg']
    dirs_temp.sort(key=lambda f: int(re.sub('\D', '', f)))
    print(dirs_temp)

    dirs.sort(key=lambda f: int(re.sub('\D', '', f)))
    #print(dirs)


    for idx,files in enumerate(tqdm(dirs)):
        # print(files)
        if idx < num_train_dataset:
            shutil.move(os.path.join(file_location,files),paths['TRAIN_PATH'])
        else:
            shutil.move(os.path.join(file_location,files),paths['TEST_PATH'])
'''
    
    with open(os.path.join(paths['TRAIN_PATH'],"_darknet.labels"),"w") as f:
        for cls in classes:
            f.write(cls + "\n")
    
    with open(os.path.join(paths['TEST_PATH'],"_darknet.labels"),"w") as f:
        for cls in classes:
            f.write(cls + "\n")
'''

'\n    \n    with open(os.path.join(paths[\'TRAIN_PATH\'],"_darknet.labels"),"w") as f:\n        for cls in classes:\n            f.write(cls + "\n")\n    \n    with open(os.path.join(paths[\'TEST_PATH\'],"_darknet.labels"),"w") as f:\n        for cls in classes:\n            f.write(cls + "\n")\n'

In [9]:
file_location = "/content/{0}".format(dataset_name)

In [10]:
convert_png_to_jpg(file_location)

No png images


In [11]:
delete_all_png(file_location)

No png images


In [12]:
check_pair_dataset(file_location,"jpg","xml")

Checking pair dataset ...
Done checking! No mismtach!


In [ ]:
rename_picFile_and_annoFile_with_order(file_location,"truck_COCO",0,"jpg","xml")

In [36]:
split_dataset(file_location,0.8)

Splitting dataset ...
['truck_COCO_2121.xml', 'truck_COCO_2835.jpg', 'truck_COCO_5771.jpg']


ValueError: ignored